<a href="https://colab.research.google.com/github/PeterRoumeliotis/AIFireSmokeDetectionResearchProject/blob/main/AIFireAndSmokeDetectionResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phylake1337/fire-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fire-dataset


In [16]:
random.seed(42)

# Finding source paths
source_dir = '/kaggle/input/fire-dataset/fire_dataset'
source_fire_dir = os.path.join(source_dir, 'fire_images')
source_non_fire_dir = os.path.join(source_dir, 'non_fire_images')

# Train and validation paths
dest_dir = 'data'
train_fire_dir = os.path.join(dest_dir, 'train', 'fire')
val_fire_dir = os.path.join(dest_dir, 'validation', 'fire')
train_non_fire_dir = os.path.join(dest_dir, 'train', 'non_fire')
val_non_fire_dir = os.path.join(dest_dir, 'validation', 'non_fire')

# Creating the directories if they don't exist
os.makedirs(train_fire_dir, exist_ok=True)
os.makedirs(val_fire_dir, exist_ok=True)
os.makedirs(train_non_fire_dir, exist_ok=True)
os.makedirs(val_non_fire_dir, exist_ok=True)

# 80% training, 20% validation
split_ratio = 0.8

# Splitting the files into each folder
def split_data(source_folder, train_folder, val_folder, split_ratio=0.8):

    file_list = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    random.shuffle(file_list)

    split_point = int(len(file_list) * split_ratio)
    train_files = file_list[:split_point]
    val_files = file_list[split_point:]

    for file_name in train_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(train_folder, file_name)
        shutil.copy(src, dst)

    for file_name in val_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(val_folder, file_name)
        shutil.copy(src, dst)

split_data(source_fire_dir, train_fire_dir, val_fire_dir, split_ratio)
split_data(source_non_fire_dir, train_non_fire_dir, val_non_fire_dir, split_ratio)

print("Data has been successfully split into training and validation")

Data has been successfully split into training and validation


In [17]:
img_width, img_height = 150, 150

batch_size = 32
epochs = 20

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

# Augmenting training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Generate validation batches
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 799 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [18]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

model_save_path = "fire_detection_cnn.keras"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.7471 - loss: 0.7984 - val_accuracy: 0.9375 - val_loss: 0.2340
Epoch 2/20
 1/24 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9062 - loss: 0.1733

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 535ms/step - accuracy: 0.9062 - loss: 0.1733 - val_accuracy: 0.9323 - val_loss: 0.2286
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.8862 - loss: 0.2745 - val_accuracy: 0.9167 - val_loss: 0.1795
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 321ms/step - accuracy: 0.8387 - loss: 0.2289 - val_accuracy: 0.9115 - val_loss: 0.1783
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.9473 - loss: 0.1450 - val_accuracy: 0.9479 - val_loss: 0.0952
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 334ms/step - accuracy: 0.9032 - loss: 0.1495 - val_accuracy: 0.9531 - val_loss: 0.0792
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.9434 - loss: 0.1337 - val_accuracy: 0.9479 - val_loss: 0.0917
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 289ms/step - accuracy: 0.9375 - loss: 0.1516 - val_accuracy: 0.9479 - val_loss: 0.0972
Epoch 9/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9708 - loss: 0.1020 - val_accuracy: 0.9688 - val_loss: 0